In [1]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import pathlib
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import cv2


In [2]:
def load_image():
    Liste =  []
    path = define_path()
    for path_dataset in path :
        for file_name in os.listdir(path_dataset)[:3]:
            if file_name.split(".")[-1].lower() in {"jpeg", "jpg", "png"}:
                x = str (path_dataset + file_name)
                Liste.append(x)
                print(file_name)               
    return Liste


def define_path():
        return ["Dataset/test/Traffic sign/","Dataset/test/Stop sign/","Dataset/test/Traffic light/","Dataset/test/Car/"]


In [3]:
def affichage(img,rec):
    r,g,b = 255 , 0,0
    img = cv2.imread(img)
    for ax in rec:   
                cv2.rectangle(img[0], (ax[-2],ax[-1]),(ax[-4],ax[-3]), (b, g, r), 1)
    
    cv2.imshow("window_name", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [18]:
def class_int_to_text(row_label):
    if row_label == 1:
        return 'Stop sign'
    if row_label == 2 :
        return 'Car'
    if row_label == 3:
        return 'Traffic sign'
    if row_label == 4:
        return 'Traffic light'
    else:
        None

In [4]:
IMAGE_PATHS = load_image()
PATH_TO_MODEL = 'exported-models_1_my_model/saved_model/'
PATH_TO_LABELS = 'training/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

45f9b594fd32feb1.jpg
01e835e68b6af93d.jpg
cd2f0a3e2c35714e.jpg
d71a9e82a6cb405e.jpg
78066a4752bc11fe.jpg
20263e31e7f52ccf.jpg
6da98e451ccc6836.jpg


In [5]:
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Done! Took 10.04129958152771 seconds


In [22]:

warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
a=0
def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

#IMAGE_PATHS = ['img.jpg']
for image_path in IMAGE_PATHS:
    a=a+1
    print('Running inference for {}... '.format(image_path), end='')
    image_np = load_image_into_numpy_array(image_path)
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    image_np_with_detections = image_np.copy()
    print("\nimage " , a)
    for i in range(len(detections['detection_scores'])):
        if detections['detection_scores'][i] > 0.23 :
            print( class_int_to_text(detections['detection_classes'][i]), detections['detection_scores'][i])

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.23,
          agnostic_mode=False)
    
    ####### choise  1: sav or  2 : show
    #1 sav
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.savefig("txt"+str(a)+".jpg")
    #2 show
    """
    img = cv2.imread(image_np_with_detections)
    cv2.imshow("window_name", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    """
plt.show()

Running inference for Dataset/test/Traffic sign/45f9b594fd32feb1.jpg... 
image  1
Car 0.30667096
Stop sign 0.29380175
Traffic sign 0.2807417
Car 0.23168895
Done
Running inference for Dataset/test/Traffic sign/01e835e68b6af93d.jpg... 
image  2
Car 0.30667096
Stop sign 0.29380175
Traffic sign 0.2807417
Car 0.23168895
Done
Running inference for Dataset/test/Stop sign/cd2f0a3e2c35714e.jpg... 
image  3
Car 0.30667096
Stop sign 0.29380175
Traffic sign 0.2807417
Car 0.23168895
Done
Running inference for Dataset/test/Stop sign/d71a9e82a6cb405e.jpg... 
image  4
Car 0.30667096
Stop sign 0.29380175
Traffic sign 0.2807417
Car 0.23168895
Done
Running inference for Dataset/test/Traffic light/78066a4752bc11fe.jpg... 
image  5
Car 0.30667096
Stop sign 0.29380175
Traffic sign 0.2807417
Car 0.23168895
Done
Running inference for Dataset/test/Car/20263e31e7f52ccf.jpg... 
image  6
Car 0.30667096
Stop sign 0.29380175
Traffic sign 0.2807417
Car 0.23168895
Done
Running inference for Dataset/test/Car/6da98e451

In [11]:
affichage(IMAGE_PATHS,detections['detection_boxes'])

In [12]:
type(detections['detection_scores'])

numpy.ndarray

In [14]:
detections['detection_scores'][1]

0.9869466